In [1]:
import ray
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray.rllib.env import PettingZooEnv
from ray.tune.registry import register_env
from spoiled_broth.rl.game_env import GameEnv
from pathlib import Path
import supersuit as ss


In [2]:
rl_model = 'PPO'
map_nr = 1

# Define reward weights
reward_weights = {
    "ai_rl_1": (0.5, 0.5),  # agent 1 is cooperative
    "ai_rl_2": (0.8, -0.2),  # agent 2 is competitive
}

In [3]:
save_path = Path(f"spoiled_broth/rl/saved_models/DTDE/{rl_model}_spoiled_broth-map_{map_nr}/")
save_path.mkdir(parents=True, exist_ok=True)

In [4]:
def env_creator(config):
    # Your existing GameEnv class goes here
    return GameEnv(**config)

#def env_creator_2(config):
#    env = GameEnv(reward_weights=reward_weights, map_nr=map_nr)
#    env = ss.pad_observations_v0(env)
#    env = ss.pad_action_space_v0(env)
#    return PettingZooEnv(env)

In [6]:
# Initialize Ray
#ray.shutdown()
#ray.init(ignore_reinit_error=True)
#
## Define the environment name
#env_name = "spoiled_broth_multiagent"

# Register the environment with RLLib
register_env("spoiled_broth", lambda config: ParallelPettingZooEnv(env_creator(config)))

# Define separate policies for each agent
def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    return f"policy_{agent_id}"


In [ ]:
# Configuration for multi-agent training
config = (
    PPOConfig()
    .environment(
        env="spoiled_broth",
        env_config={
            "reward_weights": reward_weights,
            "map_nr": map_nr
        },
        clip_actions=True
    )
    .multi_agent(
        policies={
            "shared_policy": (
                None,
                env_creator({}).observation_space("ai_rl_1"),
                env_creator({}).action_space("ai_rl_1"),
                {}
            )
        },
        policy_mapping_fn=lambda agent_id, *args, **kwargs: "shared_policy",
        policies_to_train=["shared_policy"]
    )
    .resources(num_gpus=1)  # Set to 1 if you have a GPU
    .env_runners(num_env_runners=2)
)

Weighted rewards:

Agent 1: (1.0, 0.0)

Agent 2: (1.0, 0.0)

Weighted rewards:

Agent 1: (1.0, 0.0)

Agent 2: (1.0, 0.0)

Weighted rewards:

Agent 1: (1.0, 0.0)

Agent 2: (1.0, 0.0)

Weighted rewards:

Agent 1: (1.0, 0.0)

Agent 2: (1.0, 0.0)



In [ ]:
# Build and train the algorithm
algo = config.build()

for i in range(10):  # Run for 10 iterations
    result = algo.train()
    print(f"Iteration {i}: reward={result['episode_reward_mean']}")
    
    # Save checkpoint
    if i % 5 == 0:
        checkpoint_dir = algo.save()
        print(f"Checkpoint saved in {checkpoint_dir}")
        eval_result = algo.evaluate()
        print(f"Evaluation reward: {eval_result['evaluation']['episode_reward_mean']}")

2025-05-17 12:22:58,110	WARNING deprecation.py:50 -- DeprecationWarning: `build` has been deprecated. Use `AlgorithmConfig.build_algo` instead. This will raise an error in the future!
2025-05-17 12:22:58,115	WARNING algorithm_config.py:4968 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
/home/samuel_lozano/anaconda3/envs/cooked/lib/python3.13/site-packages/ray/rllib/algorithms/algorithm.py:521: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/s

(MultiAgentEnvRunner pid=1540011) Weighted rewards:
(MultiAgentEnvRunner pid=1540011) 
(MultiAgentEnvRunner pid=1540011) Agent 1: (0.5, 0.5)
(MultiAgentEnvRunner pid=1540011) 
(MultiAgentEnvRunner pid=1540011) Agent 2: (0.8, -0.2)
(MultiAgentEnvRunner pid=1540011) 
(MultiAgentEnvRunner pid=1540012) 
(MultiAgentEnvRunner pid=1540012) 
(MultiAgentEnvRunner pid=1540012) 


(MultiAgentEnvRunner pid=1540011) 2025-05-17 12:23:03,982	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!
2025-05-17 12:23:04,211	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!
2025-05-17 12:23:05,657	WARNING util.py:61 -- Install gputil for GPU system monitoring.
(MultiAgentEnvRunner pid=1540011) 2025-05-17 12:23:05,668	ERROR actor_manager.py:187 -- Worker exception caught during `apply()`: policy_mapping_fn() missing 1 required positional argument: 'worker'
(MultiAgentEnvRunner pid=1540011) Traceback (most recent call last):
(MultiAgentEnvRunner p